In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Constants
IMAGE_SIZE = (300, 300)
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 26  # 26 classes
DATASET_PATH = "/suryas/gb_S/final_project/"

# Load dataset
def load_dataset(dataset_path):
    images = []
    labels = []
    label_mapping = {}
    
    class_folders = sorted(os.listdir(dataset_path))
    for label, category in enumerate(class_folders):
        label_mapping[label] = category
        category_path = os.path.join(dataset_path, category)
        
        for image_name in os.listdir(category_path):
            image_path = os.path.join(category_path, image_name)
            image = tf.keras.preprocessing.image.load_img(image_path, target_size=IMAGE_SIZE)
            image = tf.keras.preprocessing.image.img_to_array(image)
            images.append(image)
            labels.append(label)
    
    return np.array(images), np.array(labels), label_mapping

# Split dataset into training and testing sets
def split_dataset(images, labels):
    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test

# Data augmentation and normalization
def create_train_generator(x_train):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    train_datagen.fit(x_train)
    return train_datagen

# Define model architecture with dropout

def create_model(input_shape, num_classes):
    model = models.Sequential()
      # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Flatten
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(num_classes, activation='softmax')) 
    return model

# training process
def train_model(model, train_generator, x_train, y_train, x_test, y_test, batch_size, epochs):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    # learning rate is changed
    model.compile(optimizer = optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=len(x_train) // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))
    
    return model, history

# Save label mapping to text file
def save_label_mapping(label_mapping, file_path):
    with open(file_path, "w") as file:
        for label, category in label_mapping.items():
            file.write(f"{label}: {category}\n")

            
# Load dataset
images, labels, label_mapping = load_dataset(DATASET_PATH)

# Split dataset into training and testing sets
x_train, x_test, y_train, y_test = split_dataset(images, labels)

# Create data generator for training
train_datagen = create_train_generator(x_train)

# Define model
model = create_model(input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES)

# Train model
trained_model, history = train_model(model, train_datagen, x_train, y_train, x_test, y_test, BATCH_SIZE, EPOCHS)

# Save trained model
trained_model.save("trained_model.h5")

# Save label mapping to text file
save_label_mapping(label_mapping, "label_mapping.txt")

# test accuracy
test_loss, test_accuracy = trained_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy}")


In [ ]:
# Import necessary libraries
import cv2
import numpy as np
import math
import time
import os

# Import HandTrackingModule from cvzone
from cvzone.HandTrackingModule import HandDetector

# Initialize camera and hand detector
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)

# Constants
offset = 20  # Offset to expand hand bounding box
imgSize = 300  # Size of the square image to be saved
counter = 0  # Counter to keep track of saved images
num_images = 300  # Total number of images to capture
folder = "Signs/a2"  # Folder to save images

# Create folder if it doesn't exist
if not os.path.exists(folder):
    os.makedirs(folder)

# Main loop to capture and process images
while True:
    # Capture frame from camera
    success, img = cap.read()
    
    # Detect hands in the frame
    hands, img = detector.findHands(img, draw=True)
    
    # Process if hands are detected
    if hands:
        hand = hands[0]
        
        # Get bounding box coordinates of the hand
        x, y, w, h = hand['bbox']
        
        # Check if the hand is within the image boundaries
        if x - offset >= 0 and y - offset >= 0 and x + w + offset < img.shape[1] and y + h + offset < img.shape[0]:
            # Create a canvas to place the processed hand image
            canvas = np.ones((imgSize, imgSize, 3), np.uint8) * 255
            
            # Crop and resize the hand image to fit the canvas
            imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]
            aspectRatio = h / w
            
            # Resize and center the hand image within canvas
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                wGap = math.ceil((imgSize - wCal) / 2)
                canvas[:, wGap:wCal + wGap] = imgResize
            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                hGap = math.ceil((imgSize - hCal) / 2)
                canvas[hGap:hCal + hGap, :] = imgResize
            
            # Display cropped hand and processed canvas
            cv2.imshow("ImageCrop", imgCrop)
            cv2.imshow("Canvas", canvas)
    
    # Display original frame
    cv2.imshow("Image", img)
    
    # Wait for key press
    key = cv2.waitKey(1)
    
    # Save the processed image if 'S' key (key 32) is pressed
    if key == 32:  # 'S' key
        if counter < num_images:
            counter += 1
            # Save image with timestamp as filename
            cv2.imwrite(f'{folder}/Image_{time.time()}.jpg', canvas)
            print(counter)
        else:
            print(f"Count {num_images} reached. Change to next character.")
            break
    
    # Stop the process if 'q' key is pressed
    if key == ord("q"):
        break

# Release camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [1]:
import tkinter as tk
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import tensorflow as tf
import math
from PIL import Image, ImageTk

# Initialize camera and hand detector
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)

# Constants
offset = 30  # Offset to expand hand bounding box
imgSize = 224  # Size of the square image for gesture prediction
IMAGE_SIZE = (224, 224)  # Image size expected by the model
FPS = 10  # Frames per second for video stream

# Load the trained model for gesture prediction
model = tf.keras.models.load_model('Trained_models/latest_models/keras_model_S.h5')

# Read the class labels from the file
with open('Trained_models/latest_models/labels_W.txt', 'r') as file:
    labels_list = [line.strip() for line in file.readlines()]

def preprocess_image(img):
    # Convert the image from BGR to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Normalize pixel values to range [0, 1]
    img_normalized = img_rgb / 255.0
    # Resize the image to match the model's input size
    resized_img = cv2.resize(img_normalized, IMAGE_SIZE)
    return resized_img

def predict_gesture(hand_img):
    preprocessed_img = preprocess_image(hand_img)
    # Make a prediction using the loaded model
    predictions = model.predict(np.array([preprocessed_img]))
    # Get the index of the predicted class
    predicted_class_index = np.argmax(predictions)
    return predicted_class_index

def show_frames():
    # Read frame from camera
    success, img = cap.read()
    frame = cv2.flip(img, 1)  # Flip frame horizontally for mirror view
    
    # Detect hands in the frame
    hands, img = detector.findHands(img)
    
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']  # Get bounding box coordinates
        
        # Check if the hand is within the image boundaries
        if x - offset >= 0 and y - offset >= 0 and x + w + offset < img.shape[1] and y + h + offset < img.shape[0]:
            canvas = np.ones((imgSize, imgSize, 3), np.uint8) * 255  # Create a white canvas
            imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]  # Crop hand region
            
            # Resize and center the hand image within the canvas
            aspectRatio = h / w
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                wGap = math.ceil((imgSize - wCal) / 2)
                canvas[:, wGap:wCal + wGap] = imgResize
            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                hGap = math.ceil((imgSize - hCal) / 2)
                canvas[hGap:hCal + hGap, :] = imgResize
                
            cv2.imshow("Gesture Monitor", canvas)  # Display processed hand image
            predicted_class_index = predict_gesture(canvas)  # Predict gesture class index
            
            # Retrieve the predicted gesture label from the list
            class_label = labels_list[predicted_class_index]
            cv2.putText(frame, class_label, (frame.shape[1] - 200, 50), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0, 255, 0), 2, cv2.FILLED)
    
    # Convert frame to RGB and display in tkinter window
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame)
    img = ImageTk.PhotoImage(image=img)
    panel.img = img
    panel.config(image=img)
    
    # Schedule next frame update after specified time (milliseconds)
    panel.after(1000 // FPS, show_frames)

def quit():
    # Release camera and close all windows
    cap.release()
    cv2.destroyAllWindows()
    root.destroy()

# Create main tkinter window
root = tk.Tk()
root.title("Sign Language Prediction")

# Create label to display video stream
panel = tk.Label(root)
panel.pack(padx=10, pady=10)

# Configure window close event to call quit function
root.protocol("WM_DELETE_WINDOW", quit)

# Start displaying video frames
show_frames()

# Start the tkinter main event loop
root.mainloop()

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 32ms/step
